In [1]:
import os
import json
import scipy.io
import pandas as pd
from pathlib import Path
from helper import matfile_to_dic
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [2]:
from Nets import MLP
from Update import LocalUpdate
import argparse
import Options

In [3]:
filepath=Path('12k_DE')
dataDic=matfile_to_dic(filepath)

In [41]:
print(data)

None


In [4]:
for dicson in dataDic.values():
    del dicson['__header__']
    del dicson['__version__']
    del dicson['__globals__']
dataDic

{'B007_0.mat': {'X118_DE_time': array([-0.0027614 , -0.09632403,  0.11370459, ..., -0.07829373,
         -0.14911545,  0.02111657]),
  'X118_FE_time': array([-0.24716182,  0.14279091,  0.00328727, ...,  0.01643636,
         -0.03842   , -0.16806182]),
  'X118_BA_time': array([ 0.01553163,  0.01693994, -0.03645507, ...,  0.03178754,
         -0.03243134, -0.0668343 ]),
  'X118RPM': 1796},
 'B007_1.mat': {'X119_DE_time': array([-0.0721212 ,  0.30310395,  0.03313677, ...,  0.08820228,
         -0.19687138, -0.03898443]),
  'X119_FE_time': array([ 0.07745636,  0.04561091, -0.01109455, ..., -0.01273818,
          0.04170727,  0.02753091]),
  'X119_BA_time': array([0.07327228, 0.06932902, 0.05480332, ..., 0.05753947, 0.01569258,
         0.01488783]),
  'X119RPM': 1772},
 'B007_2.mat': {'X120_DE_time': array([-0.31447441,  0.12198878,  0.39146866, ...,  0.01900491,
          0.21067836,  0.0568523 ]),
  'X120_FE_time': array([ 0.14792727, -0.04499455,  0.10806909, ..., -0.02177818,
         

In [5]:
processed_data={0:[],1:[],2:[],3:[]}
#0:B 1:IR 2:No 3:OR
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].extend(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].extend(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].extend(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].extend(data[iname])


In [6]:
print(len(processed_data))
for i in range(4):
    print(len(processed_data[i]))

4
1947286
1948115
2182450
3418357


In [47]:
#舍弃这种方式
processed_data={0:[],1:[],2:[],3:[]}
for name,data in dataDic.items():
    if 'B' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[0].append(data[iname])
    elif 'IR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[1].append(data[iname])
    elif 'No' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[2].append(data[iname])
    elif 'OR' in name:
        for iname in data.keys():
            if 'DE' in iname:
                processed_data[3].append(data[iname])

In [57]:
#丢弃这一步
min=9999999
def comparemin(x1,x2):
    if x1>x2:
        return x2
    else:
        return x1
for data in processed_data.values():
    for iarray in data:
        ilen=len(iarray)
        min=comparemin(ilen,min)
print(min)
#min是所有文件的DE数据中的最小值

120801


In [ ]:
#这一步去掉不要
pprocessed_data={0:[],1:[],2:[],3:[]}
for name,data in processed_data.items():
    print(name)
    print(len(data))
    for idata in data:
        pprocessed_data[name].extend(idata[0:min])

这一步的目的是保持每个mat文件的采样点的一致。防止在后面进行数据划分的时候丢失某个文件的数据

In [ ]:
#去掉不要
for i in range(len(pprocessed_data)):
    print(len(pprocessed_data[i]))

In [66]:
type(processed_data[0])

list

In [7]:
splited_data=[]
label=[]
for name,data in processed_data.items():
    n=len(data)
    x=n//500
    for i in range(x):
        splited_data.append(data[i*500:(i+1)*500])
        label.append(name)

In [8]:
print(len(label))
print(len(splited_data))

18990
18990


In [9]:
data_type='iid'
clients_all=100
frac=0.1 #每次训练的客户占所有客户的比例

In [10]:
if data_type =='iid':
    num_items=int(len(label)*0.8/clients_all)
    #留下20%的数据作为实验集
    dict_users,all_idxs={},[i for i in range(len(label))]
    test_idxs=[]
    for i in range(clients_all):
        dict_users[i]=set(np.random.choice(all_idxs,num_items,replace=False))
        #随机分配数据
        all_idxs=list(set(all_idxs)-dict_users[i])
        #更新数据
    test_idxs=all_idxs

注意，dict_users={i:list_i},test_idxs=list
i代表第i个客户，list_i代表给第i个客户随机分配的数据。list中存储的是数据的下表index。数据储存再splited_data中。test_idxs储存的是测试集数据的下标。

In [11]:
print(len(test_idxs)/(len(test_idxs)+clients_all*len(dict_users[0])))

0.20484465508162192


In [12]:
net_glob=MLP(dim_in=500, dim_hidden=200, dim_out=4)
print(net_glob)
net_glob.train()

MLP(
  (layer_input): Linear(in_features=500, out_features=200, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=200, out_features=4, bias=True)
)


MLP(
  (layer_input): Linear(in_features=500, out_features=200, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=200, out_features=4, bias=True)
)

In [13]:
w_glob = net_glob.state_dict()
print(w_glob)

OrderedDict([('layer_input.weight', tensor([[ 0.0233,  0.0365,  0.0422,  ...,  0.0370, -0.0390, -0.0229],
        [ 0.0110,  0.0002,  0.0445,  ..., -0.0146,  0.0097, -0.0301],
        [ 0.0228, -0.0203, -0.0228,  ...,  0.0036,  0.0231,  0.0341],
        ...,
        [ 0.0286, -0.0129,  0.0151,  ..., -0.0281, -0.0440,  0.0240],
        [-0.0446,  0.0406, -0.0228,  ...,  0.0003,  0.0183, -0.0276],
        [ 0.0261,  0.0139,  0.0422,  ..., -0.0106,  0.0272,  0.0372]])), ('layer_input.bias', tensor([-0.0275,  0.0188, -0.0425, -0.0405,  0.0112, -0.0089,  0.0441,  0.0102,
         0.0016, -0.0009,  0.0112,  0.0014, -0.0226, -0.0397, -0.0386,  0.0376,
         0.0196, -0.0047, -0.0340,  0.0033,  0.0031, -0.0162,  0.0370,  0.0438,
         0.0430, -0.0125, -0.0038,  0.0013, -0.0048, -0.0001, -0.0391, -0.0418,
         0.0414,  0.0047,  0.0062, -0.0128,  0.0034,  0.0209,  0.0070, -0.0161,
         0.0134,  0.0323,  0.0294,  0.0113,  0.0277, -0.0111, -0.0437, -0.0126,
         0.0208, -0.0141,  

In [14]:
epochs=10

In [15]:
loss_train=[]


In [ ]:
for iter in range(epochs):
    loss_locals=[]
    w_locals = [w_glob for i in range(clients_all)]
    m=frac*clients_all
    idxs_users=np.random.choice(range(clients_all),m,replace=False)
    #选出来的是客户的编号
    for idx in idxs_users:
        
        
        
    